# 1. 전체 투표 리스트 가져오기

In [1]:
import requests

## 데이터 파라미터 조사

In [2]:
list_parameter = {
    'billName': '',
    'ageFrom': '20',
    'ageTo': '20',
    'sessionCd': '',
    'currentsCd': '',
    'currentsDt': '',
    'billKind': '',
    'chargeCommittee': '',
    'chargeCommitteeName': '',
    'orderColumn': '',
    'orderType': '',
    'lastQuery=': '',
    'strPage': '',
    'pageSize': '2239',
    'maxPage': '',
    'billNo': '',
    'allCount': '',
    'tabMenuType': 'billVoteResult',
    'searchYn': '전체',
    'idMaster': '',
    'billId': '',
    'srchHangul': '',
    'region': '',
    'searchBillNo': '',
    'searchBillName': '',
    'voteResult': ''
}
    

In [3]:
url = 'http://likms.assembly.go.kr/bill/billVoteResultListAjax.do'

In [4]:
r = requests.post(url, params=list_parameter)

In [5]:
r.status_code

200

In [6]:
json = r.json()

In [7]:
json['ageMap']

{'age': '20',
 'ageName': '제20대',
 'endDt': '20200529',
 'ord': 20,
 'sessionFrom': 343,
 'sessionTo': 366,
 'startDt': '20160530'}

## 20대 국회 전체 본회의 투표 리스트(~2019.01.05) 

In [8]:
json['allCount']

2239

## 요청 결과 받은 본회의 투표 리스트 갯수

In [9]:
len(json['resListVo'])

2239

## 요청 값과 페이지 값 실제 비교하기 

In [10]:
json['resListVo'][-1]

{'age': '20',
 'agree': 158,
 'billid': 'PRC_C1Z6N0X6S0W9I1Y5L0O6P1M6I1D0U5',
 'billkindcd': '중요동의',
 'billname': '국회상임위원회 위원정수에 관한 규칙 개정 특별위원회 구성의 건',
 'billno': '2000158',
 'currcommitte': '본회의',
 'currentscd': 1,
 'disagree': 0,
 'idmaster': 95694,
 'mtcnt': 300,
 'page': 1,
 'processdate': '2016-06-09',
 'result': '원안가결',
 'seq': 2239,
 'sessioncd': 343,
 'vtcnt': 159,
 'withdraw': 1,
 'xx': 141}

## 데이터 저장하기(csv) 

In [11]:
import csv

In [12]:
f = open('roll_call_list_20.csv', 'w') # 파일 객체 설정

In [13]:
csv_writer = csv.writer(f, delimiter=',')

In [14]:
json['resListVo'][2216]

{'age': '20',
 'agree': 209,
 'billid': 'PRC_P1F6E0M9O0X1E0Z0G3R5N1Q7T1G4U0',
 'billkindcd': '결의안',
 'billname': '2015회계연도 결산 관련 감사원에 대한 감사요구안',
 'billno': '2002009',
 'currcommitte': '예산결산특별위원회',
 'currentscd': 1,
 'disagree': 1,
 'idmaster': 100695,
 'mtcnt': 300,
 'page': 1,
 'processdate': '2016-09-02',
 'result': '원안가결',
 'seq': 2217,
 'sessioncd': 346,
 'vtcnt': 216,
 'withdraw': 6,
 'xx': 84}

In [15]:
csv_writer.writerow(json['resListVo'][0].keys())

149

In [16]:
for bill in json['resListVo']:
    csv_writer.writerow(bill.values())

# 2. 의안별 의원 투표 결과 가져오기

In [17]:
import re

In [18]:
bill_paramter = {
    'billName': '',
    'ageFrom': '20',
    'ageTo': '20',
    'age': '20',
    'sessionCd': '',
    'currentsCd': '1',
    'currentsDt': '',
    'billKind': '', 
    'chargeCommittee': '', 
    'chargeCommitteeName': '', 
    'orderColumn': '', 
    'orderType': 'ASC',
    'lastQuery': '',
    'strPage': '1',
    'pageSize': '10',
    'maxPage': '10',
    'billNo': '',
    'allCount': '1',
    'tabMenuType': 'billVoteResult',
    'searchYn': 'ABC',
    'idMaster': '',
    'billId': '',
    'srchHangul': '', 
    'region': '', 
    'searchBillNo': '', 
    'searchBillName': '', 
    'voteResult': ''
}
# billName, seesionCd, billNo, idMaster, billId, voteResult


In [19]:
bill_url = 'http://likms.assembly.go.kr/bill/billVoteResultDetail.do'

In [20]:
def extract_id(member):
    p = re.search('\d+', member)
    return p.group(0)

In [22]:
from bs4 import BeautifulSoup

In [23]:
with open('roll_call_list_20.csv', 'r') as fr:
    csv_reader = csv.DictReader(fr)
    for row in csv_reader:
        bill_paramter['billName'] = row['billname']
        bill_paramter['sessionCd'] = row['sessioncd']
        bill_paramter['idMaster'] = row['idmaster']
        bill_paramter['billNo'] = row['billno']
        bill_paramter['billId'] = row['billid']
        bill_paramter['voteResult'] = row['result']
        
        req2 = requests.post(bill_url, params=bill_paramter)
        html = req2.text
        frame = BeautifulSoup(html, 'html.parser')
        
        agree_list = frame.select('div.line01 a')
        disagree_list = frame.select('div.line02 a')
        withdraw_list = frame.select('div.line03 a')
        vote_result_list = [] # 결과값 저장 리스트 생성
        
        for member in agree_list:
            member_id = extract_id(member['href']) 
            name = member.text
            vote_result_list.append([bill_paramter['billName'],bill_paramter['billNo'], member_id, name, '찬성'])
            
        for member in disagree_list:
            member_id = extract_id(member['href']) 
            name = member.text
            vote_result_list.append([bill_paramter['billName'],bill_paramter['billNo'], member_id, name, '반대'])
            
        for member in withdraw_list:
            member_id = extract_id(member['href']) 
            name = member.text
            vote_result_list.append([bill_paramter['billName'], bill_paramter['billNo'], member_id, name, '기권'])
        
        
        break

In [24]:
vote_result_list

[['산업안전보건법 전부개정법률안(대안)', '2017834', '9770933', '강병원', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9771036', '강석진', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770279', '강창일', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770708', '경대수', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9771066', '권미혁', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770977', '권은희', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770998', '권칠승', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770927', '기동민', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770713', '김관영', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770719', '김도읍', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770997', '김두관', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770723', '김민기', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770983', '김병관', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770984', '김병욱', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9771077', '김삼화', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)', '2017834', '9770942', '김성식', '찬성'],
 ['산업안전보건법 전부개정법률안(대안)',

In [244]:
for member in agree_list:
    member_id = extract_id(member['href']) 
    name = member.text
    vote_result_list.append([member_id, name, '찬성'])

In [245]:
for member in disagree_list:
    member_id = extract_id(member['href']) 
    name = member.text
    vote_result_list.append([member_id, name, '반대'])

In [246]:
for member in withdraw_list:
    member_id = extract_id(member['href']) 
    name = member.text
    vote_result_list.append([member_id, name, '기권'])

##  결과리스트 

In [248]:
len(vote_result_list)

159

In [25]:
import time

In [28]:
with open('vote_result_list.csv', 'w') as fo:
    csv_writer = csv.writer(fo)
    csv_writer.writerow(['billname', 'billno', 'member_id', 'name', 'result'])
    with open('roll_call_list_20.csv', 'r') as fr:
        csv_reader = csv.DictReader(fr)
        vote_result_list = [] # 결과값 저장 리스트 생성
        for row in csv_reader:
            bill_paramter['billName'] = row['billname']
            bill_paramter['sessionCd'] = row['sessioncd']
            bill_paramter['idMaster'] = row['idmaster']
            bill_paramter['billNo'] = row['billno']
            bill_paramter['billId'] = row['billid']
            bill_paramter['voteResult'] = row['result']

            req2 = requests.post(bill_url, params=bill_paramter)
            html = req2.text
            frame = BeautifulSoup(html, 'html.parser')

            agree_list = frame.select('div.line01 a')
            disagree_list = frame.select('div.line02 a')
            withdraw_list = frame.select('div.line03 a')


            for member in agree_list:
                member_id = extract_id(member['href']) 
                name = member.text
                vote_result_list.append([bill_paramter['billName'],bill_paramter['billNo'], member_id, name, '찬성'])

            for member in disagree_list:
                member_id = extract_id(member['href']) 
                name = member.text
                vote_result_list.append([bill_paramter['billName'],bill_paramter['billNo'], member_id, name, '반대'])

            for member in withdraw_list:
                member_id = extract_id(member['href']) 
                name = member.text
                vote_result_list.append([bill_paramter['billName'], bill_paramter['billNo'], member_id, name, '기권'])
            time.sleep(1)
        csv_writer.writerows(vote_result_list)


In [30]:
import os

In [31]:
cwd = os.getcwd()

In [32]:
cwd

'/Users/seo/Documents/vote_data_20'

In [36]:
def main():
    
    url = 'http://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'

    """
    국회의원 현황 페이지의 경우 javascript 기반으로 작동되고 있다. selenium으로 접근 가능하지만, 좀 더 쉬운 방법으로 
    http://www.assembly.go.kr/assm/memact/congressman/memCond/memCond.do 페이지를 크롬 개발자 도구 Network를 
    보면 요청시 렌더링 되는 값들을 확인할 수 있는데 이 페이지에서는 ajax 방식을 통해 
    http://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do의 url로 데이터를 주고 받고 있다 .
    
    그리고 이 url의 페이지 소스를 보면 하단부분에 currentPage와 rowPerPage라는 파라미터가 존재하는데, 
    이를 통해 ajax 결과값을 조정하는 것을 확인할 수 있다. 
    결과적으로 rowPerPage의 값이 현재 페이지에서 보여주는 데이터들을 정의함으로, 국회의원 299명을 전체 한페이지로 불러오기 위해 
    rowPerPage 값을 300으로 정의하였다. 
    
    """
    req = requests.get(url)
    html = req.content # 요청 보낸 데이터를 받아오기

    soup = BeautifulSoup(html, 'lxml') # lxml를 파서로 사용하고, BeautifulSoup에 담기

    
    
    
    member_list = soup.select('.memberna_list dl dt a') # select 메소드는 css selector 방식읕 통해서 특정 문단을 가져올 수 있는 방법

    """
    css selector는 css 문법을 기반으로 작동한다.
    css에서는 클래스 태그의 경우 .tagname
    일반 태그의 경우 tagname으로 작동.
    그래서 위 예제에서도 각 의원들의 값을 구글 검사 기능으로 확인해보면 
    <div class='memberna_list'>
        <dl>
        <dt>
            <strong>
                <a href="javascript:jsMemPop('9770276')" title="강길부의원정보 새창에서 열림">강길부</a>
            </strong>
        </dt>
        </dl>
    </div>
    다음과 같이 되어있기 때문에 class 태그인 memberna_list는 .memberna_list로 접근 그 안의 dl은 일반태그 이기 때문에 dl 그리고 다른 태그의 경우 일반
    태그 이기 때문에 태그명으로만 접근이 가능하다. 
    그래서 select 안의 css selector 문법과 일치하는 모든 경우를 리스트 값으로 반환해준다. 
    참고 : id 태그의 경우에는 #tagname으로 접근 가능. 
    """
    
    with open('member_list.csv', 'w') as f:
        csv_writer = csv.writer(f)    
        for member in member_list:
            name = member.text # 이름값 가져오기
            id_href = member['href'] #href 값으로 접근

            pattern = re.search(r'\d+', id_href) # 문자열에서 특정 패턴의 문자들만 가져오기 (매칭되는 결과가 있으면 True, 없으면 False 반환) 예) javascript:jsMemPop('9770933') 9770933 값만 가져오기

            if pattern:
                mem_id = pattern.group(0) # 매칭되는 값 가져오기

            else:
                mem_id = None # 매칭되는 값이 없는경우 None 값 선언
            
            f = open('photo/{}.jpg'.format( mem_id), 'wb') # 파일명 지정 국회의원번호.jpg (해당 경로에 photo 라는 디렉토리 생성해야함.)
            request_photo = requests.get('http://www.assembly.go.kr/photo/{}.jpg'.format(mem_id)).content # 사진 url로 요청보내고 그 결과값 가져오기 크롬 개발자도구로 사진 url 확인가능
            f.write(request_photo) # 사진 데이터 저장해주기

            
            csv_writer.writerow([name, mem_id]) # 파일 작성해주기

        

    

    
    
    














if __name__ == "__main__":
    main()


## 국회의원 개인정보 가져오기 

In [60]:
def clean_parenthese(x):
    return x.replace('(', '').replace(')', '')

In [63]:
with open('member_list.csv', 'r') as fr:
    csv_reader = csv.reader(fr)
    with open('member_information.csv', 'w') as fw:
        
        
        col = ['name', 'mem_id', 'ch_name', 'eng_name', 'birth', 'party', 'district']
        
        csv_writer = csv.DictWriter(fw, fieldnames=col)
        csv_writer.writeheader()
        
        for name, mem_id in csv_reader:
            url = 'http://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={}'.format(mem_id)
            req = requests.get(url)
            soup = BeautifulSoup(req.text, 'html.parser')
            info_detail = soup.select('div.profile ul > li')
            profile_detail = soup.select('dl.pro_detail dd')

            info_dictionary = {
                'name': name,
                'mem_id': mem_id,
                'ch_name': clean_parenthese(info_detail[1].text.strip()),
                'eng_name': info_detail[2].text,
                'birth': info_detail[3].text,
                'party': profile_detail[0].text,
                'district': profile_detail[1].text,

            }
            csv_writer.writerow(info_dictionary)
            
        
        
            
        